In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.patches as mpatches
import numpy as np
import math

import mahotas as mh
import cv2
from PIL import Image

import copy

from skimage import io, exposure
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, opening, square, disk
from skimage.draw import disk as pdisk
from skimage.color import label2rgb, rgb2gray

from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max

from skimage.filters import threshold_local, threshold_otsu, threshold_multiotsu, rank
from skimage.filters import sobel

from scipy.stats import skew
from scipy.signal import argrelextrema

scan_path_root = "/mnt/ula/catfish/test_scans/"

path_orig = path_comm + "orig.tif"

dapi_path = path_comm + "1.tif"
homer_path = path_comm + "2.tif"
arc_path = path_comm + "3.tif"

In [ ]:
class Nucleus(): 
    def __init__(self, nucelusProps, approxEllipse,  idno):
        self.nucelusProps = nucelusProps
        self.approxEllipse = approxEllipse
        self.idno = idno

In [ ]:
def divide_region_approx_ellipse(region):
    x = 0.25*region.major_axis_length*math.sin(math.radians(-alpha))
    y = 0.25*region.major_axis_length*math.cos(math.radians(-alpha))
    approxEllipse = mpatches.Ellipse((region.centroid[1]-x,region.centroid[0]-y), region.minor_axis_length,
                                                             region.major_axis_length/2,alpha,
                           fill=False, edgecolor='green', linewidth=1)  
                new_nucleus = Nucleus(region, copy.copy(approxEllipse), len(nuclei))
                nuclei = np.append(nuclei, new_nucleus) 
                ax[1].add_patch(approxEllipse)
                
                approxEllipse = mpatches.Ellipse((region.centroid[1]+x,region.centroid[0]+y), 
                                                 region.minor_axis_length,
                                                             region.major_axis_length/2,alpha,
                           fill=False, edgecolor='green', linewidth=1)  
                new_nucleus = Nucleus(region, copy.copy(approxEllipse), len(nuclei))
                nuclei = np.append(nuclei, new_nucleus) 
                ax[1].add_patch(approxEllipse)

In [ ]:
def prepare_region_approximation(region):
    alpha = (-180/math.pi)*(region.orientation)
    if region.minor_axis_length == 0:
        ax_ratio = 0
    else:
        ax_ratio = region.major_axis_length/region.minor_axis_length
    new_nuclei = np.array([])
    #suwak + histogram do ustalania tej wartości?
    if ax_ratio * region.area > 1500:
        nuclei =
    else:
        approxEllipse = mpatches.Ellipse((region.centroid[1],region.centroid[0]), region.minor_axis_length,
                                                             region.major_axis_length,alpha,
                           fill=False, edgecolor='red', linewidth=1)  
        new_nuclei.append(Nucleus(region, copy.copy(approxEllipse), len(nuclei)))
        
    ax[1].add_patch(approxEllipse)
    
    return new_nuclei

In [ ]:
def dapi_segmentation(image_ts, footprint_size, thresh_down, thresh_up, thresh_range, sobel_g=False): 
    ratio_thre = 1.6
    
    image_ts_orig = copy.copy(image_ts)
    
    image_ts = opening(image_ts, disk(2))
    
    fig, axes = plt.subplots(nrows=3, sharex=True, sharey=True)
    ax = axes.ravel()
    fig.set_figwidth(10)
    thresh = threshold_otsu(image_ts[image_ts>0])/2
    

    local_thresh = threshold_local(image_ts, thresh_range, offset=0)
    binary_local = image_ts > local_thresh

    
    local_masked_global = np.ma.masked_array(data = binary_local, 
                                             mask = ~(image_ts > thresh), fill_value = 0).filled()

    local_masked_global = opening(local_masked_global, disk(5))
    elev_map = sobel(mh.gaussian_filter(image_ts,3))
    distance = ndi.distance_transform_edt(local_masked_global)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((footprint_size,footprint_size)),
                                labels=local_masked_global)
    markers = ndi.label(local_maxi)[0]
   
    labels = watershed(-elev_map, markers, mask=local_masked_global)

    ax[0].imshow(local_masked_global)
    ax[0].set_title("local_masked_global")
    

    label_image = label(labels, connectivity = 1)

    ax[1].imshow(image_ts_orig)
    ax[1].set_title("Labelled")
    


    ax[2].imshow(image_ts)
    ax[2].set_title("orig img")


    
    areas = np.array([])
    areas_div = np.array([])
    intensities = np.array([])
    ellipse_axes_ratios = np.array([])
    
    for region in regionprops(label_image, intensity_image=image_ts):
        areas = np.append(areas, region.area)
        intensities = np.append(intensities, region.mean_intensity)
        if region.minor_axis_length == 0:
            ax_ratio = 0
        else:
            ax_ratio = region.major_axis_length/region.minor_axis_length
        ellipse_axes_ratios = np.append(ellipse_axes_ratios, ax_ratio)

    nuclei = np.array([])
    for region in regionprops(label_image):
        #tu threshold do premyślenia
        if region.area > 10:
            prepare_region_approximation()
        nuclei = np.append(nuclei, new_nucleus)   
    
    for a in ax:
        a.set_axis_off()

    fig.tight_layout()
    plt.show()
    
    return nuclei, ellipse_axes_ratios, areas,intensities

In [ ]:
#MAIN loop to walk through img folder, looking for FOLDERS

scan_path_root = "/mnt/ula/catfish/test_scans/"

path_orig = path_comm + "orig.tif"

dapi_path = path_comm + "1.tif"
homer_path = path_comm + "2.tif"
arc_path = path_comm + "3.tif"
img = io.imread(dapi_path)